Write a solution to get all consecutive free seats.


In [0]:
from pyspark.sql.types import StructType,StructField,IntegerType,StringType,DateType
from pyspark.sql import Window
from pyspark.sql.functions import lead,col,lag,when


In [0]:
data = [
 (1, 1), (2, 0), (3, 1), (4, 1), (5, 1),
 (6, 0), (7, 1), (8, 1), (9, 0), (10, 1),
 (11, 0), (12, 1), (13, 0), (14, 1), (15, 1),
 (16, 0), (17, 1), (18, 1), (19, 1), (20, 1)
]

In [0]:
schema=StructType(
    [
        StructField("seat_id",IntegerType(),True),
        StructField("free",IntegerType(),True)
    ]
)

In [0]:
df = spark.createDataFrame(data, schema)

In [0]:
df.show()

+-------+----+
|seat_id|free|
+-------+----+
|      1|   1|
|      2|   0|
|      3|   1|
|      4|   1|
|      5|   1|
|      6|   0|
|      7|   1|
|      8|   1|
|      9|   0|
|     10|   1|
|     11|   0|
|     12|   1|
|     13|   0|
|     14|   1|
|     15|   1|
|     16|   0|
|     17|   1|
|     18|   1|
|     19|   1|
|     20|   1|
+-------+----+



#Solution1

In [0]:
w=Window.orderBy("seat_id")
df=df.withColumn("next_value",lead("free",1,0).over(w)).withColumn("prev_value",lag("free",1,0).over(w))

In [0]:
df=df.withColumn("diff1",col("next_value")-col("free")).withColumn("diff2",col("prev_value")-col("free"))
df.show()

+-------+----+----------+----------+-----+-----+
|seat_id|free|next_value|prev_value|diff1|diff2|
+-------+----+----------+----------+-----+-----+
|      1|   1|         0|         0|   -1|   -1|
|      2|   0|         1|         1|    1|    1|
|      3|   1|         1|         0|    0|   -1|
|      4|   1|         1|         1|    0|    0|
|      5|   1|         0|         1|   -1|    0|
|      6|   0|         1|         1|    1|    1|
|      7|   1|         1|         0|    0|   -1|
|      8|   1|         0|         1|   -1|    0|
|      9|   0|         1|         1|    1|    1|
|     10|   1|         0|         0|   -1|   -1|
|     11|   0|         1|         1|    1|    1|
|     12|   1|         0|         0|   -1|   -1|
|     13|   0|         1|         1|    1|    1|
|     14|   1|         1|         0|    0|   -1|
|     15|   1|         0|         1|   -1|    0|
|     16|   0|         1|         1|    1|    1|
|     17|   1|         1|         0|    0|   -1|
|     18|   1|      

In [0]:
df.filter( (col("diff1") == 0) | (col("diff2") == 0)).show()

+-------+----+----------+----------+-----+-----+
|seat_id|free|next_value|prev_value|diff1|diff2|
+-------+----+----------+----------+-----+-----+
|      3|   1|         1|         0|    0|   -1|
|      4|   1|         1|         1|    0|    0|
|      5|   1|         0|         1|   -1|    0|
|      7|   1|         1|         0|    0|   -1|
|      8|   1|         0|         1|   -1|    0|
|     14|   1|         1|         0|    0|   -1|
|     15|   1|         0|         1|   -1|    0|
|     17|   1|         1|         0|    0|   -1|
|     18|   1|         1|         1|    0|    0|
|     19|   1|         1|         1|    0|    0|
|     20|   1|         0|         1|   -1|    0|
+-------+----+----------+----------+-----+-----+



#Solution2

In [0]:
df_seat = spark.createDataFrame(data, schema)
df_seat.show()
w1=Window.orderBy('seat_id')

df1=df_seat.withColumn('prev_seat',lag('free',1).over(w1)).withColumn('next_seat',lead('free',1).over(w1))
df2=df1.filter( (col('next_seat').isin(1) & (col('free')==col('next_seat')) | (col('prev_seat').isin(1) & (col('free')==col('prev_seat'))) ))\
.select('seat_id').show()

+-------+----+
|seat_id|free|
+-------+----+
|      1|   1|
|      2|   0|
|      3|   1|
|      4|   1|
|      5|   1|
|      6|   0|
|      7|   1|
|      8|   1|
|      9|   0|
|     10|   1|
|     11|   0|
|     12|   1|
|     13|   0|
|     14|   1|
|     15|   1|
|     16|   0|
|     17|   1|
|     18|   1|
|     19|   1|
|     20|   1|
+-------+----+

+-------+
|seat_id|
+-------+
|      3|
|      4|
|      5|
|      7|
|      8|
|     14|
|     15|
|     17|
|     18|
|     19|
|     20|
+-------+



#Solution3

In [0]:
df_seat2 = spark.createDataFrame(data = data, schema = schema)
da= df_seat2.withColumn("next", lead(col("free")).over(Window.orderBy("seat_id"))).withColumn("before", lag(col("free")).over(Window.orderBy("seat_id")))
db = da.withColumn("result", when ((col("free") == col("before")) | (col("free") == col("next")), 1)\
 .otherwise(0))
dc = db.filter(col("result") == 1).select ("seat_id")
dc.show()

+-------+
|seat_id|
+-------+
|      3|
|      4|
|      5|
|      7|
|      8|
|     14|
|     15|
|     17|
|     18|
|     19|
|     20|
+-------+

